###НАЧАЛО

In [1]:
# Подключаем диск
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = '/content/drive/MyDrive/DataScience[Практика]/Дататон[хакатон]/Task3'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Подключаем библиотеки
import dlib
import os
import cv2
import xml.etree.ElementTree as pars
from google.colab.patches import cv2_imshow
import imutils

In [3]:
VAL_PATH = DRIVE_PATH + '/data/val/images/'

In [4]:
ImgNameList = os.listdir(VAL_PATH)

In [5]:
hog = cv2.HOGDescriptor() 
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector()) 

###EVAL

In [35]:
import cv2
import imutils
from google.colab.patches import cv2_imshow
from numpy import arange, array
import numpy as np


#модуль для детектирования

# если вы используте модель для детектирования добавьте ее здесь:
#################################################################
hog = cv2.HOGDescriptor() 
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector()) 
#################################################################

def area_of_intersection(coords1, coords2):
    x1, y1, x2, y2 = coords1
    x3, y3, x4, y4 = coords2

    # границы области пересечения
    left = max(x1, x3)  # левая
    bottom = max(y1, y3)  # нижняя
    right = min(x2, x4)  # правая
    top = min(y2, y4)  # верхняя

    width = right - left
    height = top - bottom

    if width <= 0 or height <= 0:
        area = 0
    else:
        area = width * height
    return area


def area(coords):
    x1, y1, x2, y2 = coords
    return (x2 - x1) * (y2 - y1)


def detector(image):
  height, width, channels = image.shape
  scale_size = 1.75
  size = int(width // scale_size), int(height // scale_size)
  image = cv2.resize(image, size)
  # kernel = np.array([[-0.2,-0.2,-0.2], [-0.2,3,-0.2], [-0.2,-0.2,-0.2]])
  regions, _ = hog.detectMultiScale(image, 
                                 scale=1.01, 
                                 winStride=(2, 2))

  rects = []
  image = cv2.resize(image,  (width, height))
  
  for (x, y, w, h) in regions:
      errs = []
      for ind, (x2, y2, w2, h2) in enumerate(rects):
        if (x, y, x + w, y + h) == (x2, y2, x2 + w2, y2 + h2):
          continue
        area1 = area((x, y, x + w, y + h))
        area2 = area((x2, y2, x2 + w2, y2 + h2))
        intersection_area = area_of_intersection((x, y, x + w, y + h), (x2, y2, x2 + w2, y2 + h2))
        if intersection_area / area1 > 0.6 or intersection_area / area2 > 0.6:
          errs.append(ind)
          print('Deleted')

      rects = [x for i, x in enumerate(rects) if i not in errs]

      rects.append([x, y, x + w, h + y])

  rects = list(map(lambda x:list(map(int, (x[0] * scale_size, x[1] * scale_size, x[2] * scale_size, x[3] * scale_size))), rects))
  for rect in rects:
    x2, y2, w2, h2 = rect
    cv2.rectangle(image, (x2, y2),  
                    (x2 + w2, y2 + h2),  
                    (0, 255, 0), 2) 
  cv2_imshow(image)
  return rects

###HELPERS

In [7]:
import cv2
import os
import xml.etree.ElementTree as pars

def load_images_from_folder(folder):

    im_list = []

    for filename in os.listdir(folder + "/images/"):
        img = cv2.imread(os.path.join(folder + "/images/" + filename))
        full_name = filename.split('.')
        part_name = full_name[0]

        if not img is None:
                   im_list.append((part_name, img))

    return im_list

def load_objects(im_list, folder):

    object_list = []

    for pic in im_list:
        part_name = pic[0]
        img = pic[1]

        e = pars.parse(folder + '/annotations/xmls/' + part_name + '.xml')

        root = e.getroot()

        objects = []
        for object in root.findall('object'):
            name = object.find('name').text
            for box in object.findall('bndbox'):
                points = [int(box.find('xmin').text),
                                   int(box.find('ymin').text),
                                   int(box.find('xmax').text),
                                   int(box.find('ymax').text)]
            objects.append(points)
        object_list.append([img, part_name, objects])

    # print(object_list[0][2][0][1])
    return object_list
    # return object_list

###MAIN

In [37]:
import cv2


def load_data():
    try:
        image_dir_test = DRIVE_PATH + "/data/train/"
        test_image_list = load_images_from_folder(image_dir_test)
        test_object_list = load_objects(test_image_list, image_dir_test)
    except Exception as e:
      print(e)
      test_object_list = []
    return test_object_list


def predict(frame):
    predicted_label = detector(frame)
    return predicted_label


def iou(points, predicted_points):
    """IoU = Area of overlap / Area of union
    Площадь пересечеения 2 знаков / общая площадь 2 знаков
    Значение выше 0.5 считаестя удачным детектированием знака
    """

    xA = max(points[0], predicted_points[0])
    yA = max(points[1], predicted_points[1])
    xB = min(points[2], predicted_points[2])
    yB = min(points[3], predicted_points[3])

    # x1, y1, xw1, yh1 = points
    # x2, y2, xw2, yh2 = predicted_points
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles

    boxAArea = (points[2] - points[0] + 1) * (points[3] - points[1] + 1)
    boxBArea = (predicted_points[2] - predicted_points[0] + 1) * (predicted_points[3] - predicted_points[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou


def check_prediction(test_image_label, predict_image_label):
    """ Проверка детектирования на изображении.
        Двойной проход по изображениям знака, обеспечивающий сравнение каждого существующего
        знака со всеми предсказанными по очереди и оперделяющий знак при наличии совпадения.
        Также при отсутствии совпадения определяющий что данный знак отсутствует и формирующй
        массив ложных совпадений
    """

    true_predicted = []
    false_predicted = []

    for p_obj in predict_image_label:

        predicted = []

        for obj in test_image_label:
            iou_res = iou(obj, p_obj)
            predicted.append(iou_res)

        pred = None
        for i in predicted:
            if i > 0.35:
                pred = True
                break
            else:
                pred = False

        if pred == True:
            true_predicted.append(1)
        else:
            false_predicted.append(1)



    if len(true_predicted) >= len(test_image_label):
        predicted_count = len(test_image_label)
    else:
        predicted_count = len(true_predicted)

    local_acc = predicted_count / len(test_image_label)

    false_positive = len(false_predicted) / len(test_image_label)

    return local_acc, false_positive


def get_misclassified_images(test_images):
    """ Проверка массива изображений на совпадение
    Результат: среднее точности и среднееточности срабатывания"""
    local_acc_summ = 0.
    false_positive_summ = 0.
    count = 0.

    for frame in test_images:
        # cv2.imshow("frame",frame)
        # cv2.waitKey(0)
        count += 1

        test_image_label = frame[2]
        predict_image_label = predict(frame[0])

        local_acc, local_acc_false_positive = check_prediction(test_image_label, predict_image_label)

        local_acc_summ += local_acc
        false_positive_summ += local_acc_false_positive

    return (local_acc_summ / count, false_positive_summ / count)


test_object_list = load_data()
# print(test_object_list)


MISCLASSIFIED = get_misclassified_images(test_object_list)
print("Точность: {}, Ложное предсказание (false positive): {}".format(MISCLASSIFIED[0], MISCLASSIFIED[1]))
fin_acc = MISCLASSIFIED[0]-MISCLASSIFIED[1]
if fin_acc < 0:
    fin_acc = 0
print("Финальная точность: {}".format(fin_acc))

Output hidden; open in https://colab.research.google.com to view.